In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

In [ ]:
!pip install nltk
!pip install optuna

import pandas as pd
import random
from openai import OpenAI
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.translate.meteor_score import meteor_score
from itertools import combinations
import optuna

# Ensure the necessary NLTK resources are downloaded
nltk.download('wordnet')

API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125'

In [ ]:
credit_df = pd.read_csv('credit_new.csv')
credit_df = credit_df.fillna('Unknown')
credit_df.drop('Unnamed: 0', axis=1, inplace=True)

credit_df['SeriousDlqin2yrs'] = credit_df['SeriousDlqin2yrs'].replace(0, 'will not experience financial distress')
credit_df['SeriousDlqin2yrs'] = credit_df['SeriousDlqin2yrs'].replace(1, 'will experience financial distress')

# Remove the column and store it
column_to_move = credit_df.pop('SeriousDlqin2yrs')

# Add the column to the end of the DataFrame
credit_df['SeriousDlqin2yrs'] = column_to_move

print(credit_df.head(5))

tuning_set = credit_df.sample(n = 20, random_state = 1) # adjust tuning set size here
test_set = credit_df.drop(tuning_set.index)

percentage_yes = (tuning_set['SeriousDlqin2yrs'] == 'will experience financial distress').mean()
print(tuning_set['SeriousDlqin2yrs'].unique())
print(percentage_yes)

row_series = tuning_set.iloc[0][:-1]

all_features = ''
for feature, value in row_series.items():
    all_features += feature + ', '

print(all_features)

selected_rows_1 = credit_df.loc[credit_df['SeriousDlqin2yrs'] == 'will experience financial distress'].head(5)

selected_rows_2 = credit_df.loc[credit_df['SeriousDlqin2yrs'] == 'will not experience financial distress'].head(15)

# Combine the two DataFrames into a new DataFrame
combined_df = pd.concat([selected_rows_1, selected_rows_2], ignore_index=True)

samples = []
targets = []
for i in range( len(combined_df) ):
    row_series = combined_df.iloc[i][:-1]
    target = combined_df.iloc[i]['SeriousDlqin2yrs']

    overall_feature_levels = ''
    for feature, value in row_series.items():
        feature_level = (f"Feature: {feature}, Value: {value}")
        overall_feature_levels += feature_level + '; '

    samples.append(overall_feature_levels)
    targets.append(target)

print(samples)
print(targets)

In [ ]:
def mean_pairwise_meteor(prompts):
    """
    Compute the mean pairwise METEOR score among a list of prompts.

    :param prompts: List of generated text outputs
    :return: Mean METEOR score across all prompt pairs
    """
    # Tokenize each prompt into a list of words
    tokenized_prompts = [p.split() for p in prompts]

    pairs = list(combinations(tokenized_prompts, 2))  # Get all unique pairs
    scores = [meteor_score([p1], p2) for p1, p2 in pairs]  # Compute METEOR for each pair

    return sum(scores) / len(scores) if scores else 0

# # Example usage
# prompts = ["this is a test sentence", "this is an example test", "another test case"]
# mean_score = mean_pairwise_meteor(prompts)
# print("Mean Pairwise METEOR Score:", mean_score)



def mean_pairwise_cosine_similarity(arrays):
    # Stack arrays into a single matrix
    matrix = np.vstack(arrays)

    # Compute cosine similarity matrix
    sim_matrix = cosine_similarity(matrix)

    # Exclude diagonal elements (self-similarity)
    n = len(arrays)
    mask = np.ones((n, n), dtype=bool)
    np.fill_diagonal(mask, 0)

    # Compute the average pairwise similarity
    avg_similarity = sim_matrix[mask].mean()

    return avg_similarity

# # Example usage
# arrays = [np.random.rand(10) for _ in range(5)]  # List of 5 random 10-dimensional vectors
# avg_sim = mean_pairwise_cosine_similarity(arrays)
# print("Mean Pairwise Cosine Similarity:", avg_sim)



def ADO(num_prompts, cos_sim, lexical_sim):

    few_shot_pairs = f'''
        - Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
        - Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
        - Financial Profile: '{samples[2]}'; Will the person experience financial distress in the next two years: '{targets[2]}'
        - Financial Profile: '{samples[3]}'; Will the person experience financial distress in the next two years: '{targets[3]}'
        - Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'
    '''

    ADO_prompt = f"""
    You are given a set of feature-value pairs that represents a person's financial profile. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich (make more informative) this profile. The goal is of the algorithm is to perform a thorough data engineering among the feature values, so that it is easier for a LLM to distinguish whether the person is likely to experience financial distrees in the next 2 years. Below are some sample financial profiles as refrences.

    Examples:
    - Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
    - Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
    - Financial Profile: '{samples[5]}'; Will the person experience financial distress in the next two years: '{targets[5]}'
    - Financial Profile: '{samples[6]}'; Will the person experience financial distress in the next two years: '{targets[6]}'
    - Financial Profile: '{samples[7]}'; Will the person experience financial distress in the next two years: '{targets[7]}'
    - Financial Profile: '{samples[8]}'; Will the person experience financial distress in the next two years: '{targets[8]}'
    - Financial Profile: '{samples[9]}'; Will the person experience financial distress in the next two years: '{targets[9]}'
    - Financial Profile: '{samples[10]}'; Will the person experience financial distress in the next two years: '{targets[10]}'
    - Financial Profile: '{samples[11]}'; Will the person experience financial distress in the next two years: '{targets[11]}'
    - Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'

    For each step of the algorithm, number and then start it on a new line; you MUST MUST start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
    Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform PCA. Do NOT perform standardization; Do NOT perform one-hot encoding; Do NOT perform vector generations. Do NOT perform similarity checking. ONLY propose things that a LLM can do on its own!
    """

    search_rounds = 3
    prompt_performances = {}

    for s in range(search_rounds):

        total_try = 0
        prompts = []
        prompt_embeddings = []
        prompt_cos_dict = {}
        mean_cos_sim = 1.0

        while mean_cos_sim > cos_sim:

            for i in range(num_prompts):
                completion = client.chat.completions.create(
                    model = model_id, temperature = 1, max_tokens = 1000,

                    messages=[{"role": "system", "content": "Please come up with a step-by-step, very detailed, clear, and very novel algorithm for enriching and then reformulating a person's financial profile. Return the steps ONLY!!! NO more than 6 steps. You MUST NOT directly include whether the person will expereince financial distress in the next 2 years within the generated text!!!"},
                                {"role": "user", "content": ADO_prompt}],
                    timeout = 1200)

                candidate_prompt = completion.choices[0].message.content
                prompts.append(candidate_prompt)

                response = client.embeddings.create(
                    input=candidate_prompt,
                    model="text-embedding-3-small",
                )

                candidate_prompt_embedding = np.array(response.data[0].embedding)
                prompt_embeddings.append(candidate_prompt_embedding)

                # ADO_prompt += '\n\n\n Please generate each step to be completely different in wording and semantics from this one: \n\n' + candidate_prompt + '\n\n\n'


            total_try += 1
            mean_meteor = mean_pairwise_meteor(prompts)
            mean_cos_sim = mean_pairwise_cosine_similarity(prompt_embeddings)

            # Define min and max based on typical ranges (adjustable)
            meteor_min, meteor_max = 0.0, 1.0  # METEOR is usually in [0,1]
            cos_min, cos_max = 0.5, 1.0  # Cosine similarity often ranges [0.5,1] in text similarity tasks

            # Min-max normalization
            meteor_norm = (mean_meteor - meteor_min) / (meteor_max - meteor_min)
            cos_norm = (mean_cos_sim - cos_min) / (cos_max - cos_min)

            prompt_cos_dict[tuple(prompts)] = meteor_norm + cos_norm

            if mean_cos_sim <= cos_sim and mean_meteor <= lexical_sim:
                print('Qualifying prompts generated!')
                break

            if total_try > 5:
                prompts = min(prompt_cos_dict, key=prompt_cos_dict.get)
                break

            prompts = []
            prompt_embeddings = []


        # Now we have the candidate prompts, start tuning set evaluation:
        system_msg = 'Please serve as a binary classifier on whether the person is likely to experience financial distress within the next 2 years, based on his/her current financial profile.'

        for candidate_prompt in prompts:
            print(candidate_prompt)
            print()

            # Regular expression to split the text into individual algorithms
            algorithm_pattern = r"###------- Step \d: [\w\s]+"

            # Split the text based on the pattern
            split_text = re.split(algorithm_pattern, candidate_prompt)

            # Extract the algorithm headers (for identification)
            headers = re.findall(algorithm_pattern, candidate_prompt)

            # Removing the first empty string from the split if exists (because of the split at the start)
            split_text = [t.strip() for t in split_text if t.strip()]

            # Create a dictionary where each algorithm is stored separately
            algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

            # Display each algorithm separately
            steps = []
            for header, content in algorithms.items():
                steps.append( [header, content] )
                # print(f"{header}:\n{content}\n")

            right_count_pos = 0
            right_count_neg = 0
            compressed_right_count_pos = 0
            compressed_right_count_neg = 0
            total_pos = 0
            total_neg = 0
            total = 0

            # you may also set range smaller for a subset
            for i in range( len(tuning_set) ):
                row_series = tuning_set.iloc[i][:-1]
                target = tuning_set.iloc[i]['SeriousDlqin2yrs']

                overall_feature_levels = ''
                for feature, value in row_series.items():
                    feature_level = (f"Feature: {feature}, Value: {value}")
                    overall_feature_levels += feature_level + '; '


                # original_prompt = (
                #     "You are given a person's financial profile with each feature-value pair presented."
                #     "Please determine whether the person is likely to experience a financial distress within the next 2 years. First write a step-by-step reasoning paragraph, and then output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
                #     "Below is the person's financial profile: \n\n"
                #     f"{overall_feature_levels}"
                # )

                # original_prompt = (
                #     f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
                #     f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
                # )

                original_prompt = (
                    f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
                    f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
                    f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
                )

                completion = client.chat.completions.create(
                        model = model_id, temperature = 0, seed = 0,
                        messages=[{"role": "system", "content": system_msg},
                                    {"role": "user", "content": original_prompt }],
                        timeout = 1200)

                original_pred = completion.choices[0].message.content
                # print(original_pred)
                # print()

                total += 1
                if target == 'will experience financial distress':
                    total_pos += 1
                else:
                    total_neg += 1

                if target in original_pred:
                    if (target == 'will experience financial distress'):
                        right_count_pos += 1
                    if (target == 'will not experience financial distress'):
                        right_count_neg += 1



                overall_with_steps = "Original person's financial profile: " + overall_feature_levels + '\n\n'
                for i in range(len(algorithms)):

                    reformulation_prompt = f"Please thoroughly enrich and reformulate the person's financial profile based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nprofile to enrich and reformulate: {overall_with_steps}\n\nYou MUST return the reformulation of the profile ONLY!!!"

                    completion = client.chat.completions.create(
                            model = model_id, temperature = 0.0, max_tokens = 512,

                            messages=[{"role": "system", "content": "Please enrich and reformulate the person's profile to be extremely informative and detailed for LLM to better recognize the individuals that will be in financial distress within the next 2 years. You are allowed to fill in unspecified information based on your domain expertise!"},
                                        {"role": "user", "content": reformulation_prompt}],
                            timeout = 1200)

                    reformulated_history = completion.choices[0].message.content
                    # print('Step ' + str(i) + ' Results: ')
                    # print(reformulated_history)
                    overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
                    # print()
                    # print()

                # print(overall_with_steps)
                # print()

                ado_prompt = (
                    f"Given the following person's financial profile: \n\n {overall_with_steps};\n\n"
                    f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
                    f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
                )

                compressed_completion = client.chat.completions.create(
                        model = model_id, temperature = 0, seed = 0,
                        messages=[{"role": "system", "content": system_msg},
                                    {"role": "user", "content": ado_prompt}],
                        timeout = 1200)

                compressed_pred = compressed_completion.choices[0].message.content


                if target in compressed_pred:
                    if (target == 'will experience financial distress'):
                        compressed_right_count_pos += 1
                    if (target == 'will not experience financial distress'):
                        compressed_right_count_neg += 1


                # print(converted_feature_levels)
                print(right_count_pos, right_count_neg)
                print(compressed_right_count_pos, compressed_right_count_neg)
                print()

                # perform early-stopping
                if total % 50 == 0 or total == credit_df.shape[0]:
                    accuracy = (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5
                    compressed_accuracy = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
                    print('Accuracy: ', accuracy)
                    print('Compressed Accuracy: ', compressed_accuracy)
                    print()

                    if compressed_accuracy <= accuracy:
                        break

            prompt_performances[candidate_prompt] = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
            print( prompt_performances[candidate_prompt] )

        prior_exp = ""
        for k,v in prompt_performances.items():
            prior_exp += '\n\n' + 'Algorithm: ' + k + '\n\n' + 'Score: ' + str(v)


        ADO_prompt = f"""
        You are given a set of feature-value pairs that represents a person's financial profile. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich (make more informative) this profile. The goal is of the algorithm is to perform a thorough data engineering among the feature values, so that it is easier for a LLM to distinguish whether the person is likely to experience financial distrees in the next 2 years. Below are some sample financial profiles as refrences.

        Examples:
        - Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
        - Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
        - Financial Profile: '{samples[5]}'; Will the person experience financial distress in the next two years: '{targets[5]}'
        - Financial Profile: '{samples[6]}'; Will the person experience financial distress in the next two years: '{targets[6]}'
        - Financial Profile: '{samples[7]}'; Will the person experience financial distress in the next two years: '{targets[7]}'
        - Financial Profile: '{samples[8]}'; Will the person experience financial distress in the next two years: '{targets[8]}'
        - Financial Profile: '{samples[9]}'; Will the person experience financial distress in the next two years: '{targets[9]}'
        - Financial Profile: '{samples[10]}'; Will the person experience financial distress in the next two years: '{targets[10]}'
        - Financial Profile: '{samples[11]}'; Will the person experience financial distress in the next two years: '{targets[11]}'
        - Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'

        For each step of the algorithm, number and then start it on a new line; you MUST MUST start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
        Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform PCA. Do NOT perform standardization; Do NOT perform one-hot encoding; Do NOT perform vector generations. Do NOT perform similarity checking. ONLY propose things that a LLM can do on its own!
        """

        ADO_prompt += '\n\nBelow are some algorithm-score pairs for you to refer to prior to generation:' + '\n' + prior_exp
        print(ADO_prompt)
        print()

    prompt = max(prompt_performances, key=prompt_performances.get)
    return [prompt, prompt_performances[prompt]]


# # example run
# output = ADO(2, 0.7, 0.3)

# print(output[0])
# print()
# print(output[1])

In [ ]:
# perform Bayes Opt.
best_prompt_per_round = {}

def objective(trial):
    num_prompts = trial.suggest_int('num of prompts', 2, 4)
    cos_sim = trial.suggest_float('Mean Cosine Sim among prompts', 0.6, 0.95, step = 0.05)
    lexical_sim = trial.suggest_float('Mean Meteor among prompts', 0.2, 0.5, step = 0.05)

    outputs = ADO(num_prompts, cos_sim, lexical_sim)
    best_prompt_per_round[ outputs[0] ] = outputs[1]
    print(outputs[1])

    return outputs[1] # use val accuracy to search for best hyper-param set


# start hyper-param tuning
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 8)

final_prompt = max(best_prompt_per_round, key=best_prompt_per_round.get)

num_prompts = study.best_params['num of prompts']
cos_sim = study.best_params['Mean Cosine Sim among prompts']
lexical_sim = study.best_params['Mean Meteor among prompts']

## Performance Evaluation

In [ ]:
few_shot_pairs = f'''
    - Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
    - Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
    - Financial Profile: '{samples[2]}'; Will the person experience financial distress in the next two years: '{targets[2]}'
    - Financial Profile: '{samples[3]}'; Will the person experience financial distress in the next two years: '{targets[3]}'
    - Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'
'''

system_msg = 'Please serve as a binary classifier on whether the person is likely to experience financial distress within the next 2 years, based on his/her current financial profile.'


print(final_prompt)
print()

# Regular expression to split the text into individual algorithms
algorithm_pattern = r"###------- Step \d: [\w\s]+"

# Split the text based on the pattern
split_text = re.split(algorithm_pattern, final_prompt)

# Extract the algorithm headers (for identification)
headers = re.findall(algorithm_pattern, final_prompt)

# Removing the first empty string from the split if exists (because of the split at the start)
split_text = [t.strip() for t in split_text if t.strip()]

# Create a dictionary where each algorithm is stored separately
algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

# Display each algorithm separately
steps = []
for header, content in algorithms.items():
    steps.append( [header, content] )
    # print(f"{header}:\n{content}\n")

right_count_pos = 0
right_count_neg = 0
compressed_right_count_pos = 0
compressed_right_count_neg = 0
total_pos = 0
total_neg = 0
total = 0

# you may also set range smaller for a subset
for i in range( len(test_set) ):
    row_series = test_set.iloc[i][:-1]
    target = test_set.iloc[i]['SeriousDlqin2yrs']

    overall_feature_levels = ''
    for feature, value in row_series.items():
        feature_level = (f"Feature: {feature}, Value: {value}")
        overall_feature_levels += feature_level + '; '


    # original_prompt = (
    #     "You are given a person's financial profile with each feature-value pair presented."
    #     "Please determine whether the person is likely to experience a financial distress within the next 2 years. First write a step-by-step reasoning paragraph, and then output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
    #     "Below is the person's financial profile: \n\n"
    #     f"{overall_feature_levels}"
    # )

    # original_prompt = (
    #     f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
    #     f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
    # )

    original_prompt = (
        f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
        f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
        f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
    )

    completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,
            messages=[{"role": "system", "content": system_msg},
                        {"role": "user", "content": original_prompt }],
            timeout = 1200)

    original_pred = completion.choices[0].message.content
    # print(original_pred)
    # print()

    total += 1
    if target == 'will experience financial distress':
        total_pos += 1
    else:
        total_neg += 1

    if target in original_pred:
        if (target == 'will experience financial distress'):
            right_count_pos += 1
        if (target == 'will not experience financial distress'):
            right_count_neg += 1



    overall_with_steps = "Original person's financial profile: " + overall_feature_levels + '\n\n'
    for i in range(len(algorithms)):

        reformulation_prompt = f"Please thoroughly enrich and reformulate the person's financial profile based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nprofile to enrich and reformulate: {overall_with_steps}\n\nYou MUST return the reformulation of the profile ONLY!!!"

        completion = client.chat.completions.create(
                model = model_id, temperature = 0.0, max_tokens = 512,

                messages=[{"role": "system", "content": "Please enrich and reformulate the person's profile to be extremely informative and detailed for LLM to better recognize the individuals that will be in financial distress within the next 2 years. You are allowed to fill in unspecified information based on your domain expertise!"},
                            {"role": "user", "content": reformulation_prompt}],
                timeout = 1200)

        reformulated_history = completion.choices[0].message.content
        # print('Step ' + str(i) + ' Results: ')
        # print(reformulated_history)
        overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
        # print()
        # print()

    # print(overall_with_steps)
    # print()

    ado_prompt = (
        f"Given the following person's financial profile: \n\n {overall_with_steps};\n\n"
        f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
        f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
    )

    compressed_completion = client.chat.completions.create(
            model = model_id, temperature = 0, seed = 0,
            messages=[{"role": "system", "content": system_msg},
                        {"role": "user", "content": ado_prompt}],
            timeout = 1200)

    compressed_pred = compressed_completion.choices[0].message.content


    if target in compressed_pred:
        if (target == 'will experience financial distress'):
            compressed_right_count_pos += 1
        if (target == 'will not experience financial distress'):
            compressed_right_count_neg += 1


    # print(converted_feature_levels)
    print(right_count_pos, right_count_neg)
    print(compressed_right_count_pos, compressed_right_count_neg)
    print()

    if total % 30 == 0 or total == credit_df.shape[0]:
        accuracy = (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5
        compressed_accuracy = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
        print('Accuracy: ', accuracy)
        print('Compressed Accuracy: ', compressed_accuracy)
        print()